<a href="https://colab.research.google.com/github/letteredMelody/colab_python/blob/main/QRLU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Не забываем загрузить библиотеки:

In [ ]:
# Библиотека для работы с матрицами
import numpy as np 

# Алгоритмы линейной алгебры
import scipy.linalg as sla

# Библиотека для работы с разреженными матрицами
import scipy.sparse as sps

# Алгоритмы линейной алгебры для разреженных матриц
import scipy.sparse.linalg as spla

# Графическая библиотека
import matplotlib.pyplot as plt

# Позволяет отрисовывать графики и изображения прямо в ноутбуке, а не в отдельном окне. Жизненно важная вещь!
%matplotlib inline

# Теория

Информация в этой клетке с википедии, при желании — переходите по ссылкам и читайте полную версию.

## [Нормы](https://ru.wikipedia.org/wiki/Норма_(математика))
Вы знаете, что в векторном пространстве над полем вещественных или комплексных чисел есть понятие длины вектора — квадратный корень из скалярного произведения вектора на самого себя.  
Норма обобщает понятие длины вектора. Это функционал $||\cdot||: V \to \mathbb{R}_+$, обладающий следующими свойствами:

1. $||x|| = 0 \Rightarrow x = 0$;
2. $\forall x,y \in V,\ ||x+y|| \le ||x|| + ||y||$ (неравенство треугольника);
3. $\forall \alpha \in \mathbb{R} (\mathbb{C})\ \forall x \in V,\ ||\alpha x|| = |\alpha|\cdot ||x||$.

У некоторых норм есть общепринятые названия. Когда говорят об $l_p$-норме, имеют ввиду отображение
$$
||\cdot||_p: V \to \mathbb{R}_+,\quad ||(x_1, \ldots, x_n)||_p = (|x_1|^p + \ldots + |x_n|^p)^{\frac{1}{p}}
$$

В частности, $l_2$-норма — это длина вектора, $l_1$-норма — это сумма модулей координат, норма $l_\infty$ равна максимуму из модулей координат.

Норма матрицы — это просто её норма как вектора пространства $K^{m \times n}$ (где $K \in \{\mathbb{R}, \mathbb{C}\}$).  
Есть ещё операторные нормы, которые определяются через векторную норму:
$$
||\varphi|| = \sup\limits_{||x|| = 1} ||\varphi(x)||.
$$
Нас тут интересует только то, что используя любую норму, заданную на векторах, можно определить соответствующую операторную норму, которая в свою очередь определяет матричную норму.  
Таким образом определённая матричная норма (через норму вектора) называется согласованной с соответствующей векторной нормой.

## [$LU$-разложение](https://ru.wikipedia.org/wiki/LU-разложение#Вывод_формулы)
$LU$-разложение матрицы $A$ — это представление этой матрицы в виде произведения $A = LU$ нижнетреугольной матрицы $L$ и верхнетреугольной матрицы $U$.  
Это разложение важно при решении линейных систем. Если вы знаете такое разложение матрицы $A$, то решение СЛУ
$$Ax = b \Leftrightarrow LUx = b$$
сводится к решению двух систем
$$Ly = b \text{ и } Ux = y$$
c треугольными матрицами (а значит решаемыми одним прямым или обратным проходом).

## [$QR$-разложение](https://ru.wikipedia.org/wiki/QR-разложение)
$QR$-разложение матрицы $A$ — это представление этой матрицы в виде произведения $A = QR$ ортогональной (или унитарной) матрицы $Q$ и верхнетреугольной матрицы $R$.  

В силу того, что $Q^*Q = I$, при известном $QR$-разложении матрицы $A$ решение СЛУ 
$$Ax = b \Leftrightarrow QRx = b$$
сводится к решению системы
$$Rx = Q^* b$$
с треугольной матрицей.

**Задание 0 (1 балл) (теоретическое)** Найдите (по определению или в википедии), чему равны матричные нормы $l_1$ и $l_\infty$ и докажите полученные формулы.

Векторной $l_1$-норме соответствует матрица $\max _{j}\sum _{i}|a_{{ij}}|$. По определению:

$$
||A||_1 = \sup\limits_{||x||_1 = 1} ||Ax||_1 = \sup\limits_{x=1} Ax = A
$$

Векторной $l_2$-норме соответствует так называемая спектральная норма матрицы, для матрицы $A$ равная корню из максимального собственного числа матрицы $A^*A$ (где $A^*$ это сопряжённая матрица):
$$
||A||_2 = \sup\limits_{||x||_2 = 1} ||Ax||_2 = \sup\limits_{(x,x)=1} \sqrt{(Ax,Ax)} = \sqrt{\lambda_{max}(A^*A)}
$$

Векторной $l_\infty$ норме соответствует матрица $\max _{i}\sum _{j}|a_{{ij}}|$. По определению:

$$
||A||_\infty = \sup\limits_{||x||_\infty = 1} ||Ax||_\infty = \sup\limits_{(x, \ldots, x) =1} (|x_1|^\infty + \ldots + |x_n|^\infty)^{\frac{1}{\infty}} = A^*
$$

## Вычислительные особенности

С точки зрения математики матричные разложения являются точными: произведение сомножителей всегда равняется исходной матрице $A$. К сожалению, на практике этому часто мешает вычислительная погрешность. 

Для $LU$ разложения $l_2$-норма ошибки ошибки $||\delta A|| = ||A - LU||$ удовлетворяет следующей оценке:

$$||\delta A|| \leqslant ||L|| \cdot ||U|| \cdot O(\varepsilon_{machine})$$

А нормы $L$ и $U$ могут быть совсем нехорошими.

**Задание 1 (1 балл)** Рассмотрим следующее LU-разложение:

$$
\begin{pmatrix}
10^{-20} & 1\\
1 & 1
\end{pmatrix} = \begin{pmatrix}
1 & 0\\
10^{20} & 1
\end{pmatrix}\cdot\begin{pmatrix}
10^{-20} & 1\\
0 & 1 - 10^{20}
\end{pmatrix}
$$

Перемножьте полученные матрицы $L$ и $U$. А теперь перемножьте такие же матрицы, только после всех единиц поставьте десятичные точки. Изменился ли ответ? Как вам кажется, почему?

In [ ]:
L = np.matrix([[1, 0], [10**20, 1]])
U = np.matrix([[10 ** -20, 1], [0, 1 - 10**20]])

print(L * U)

L_dot = np.matrix([[1., 0], [10**20, 1.]])
U_dot = np.matrix([[10 ** -20, 1.], [0, 1 - 10**20]])

print("\n", L_dot * U_dot)

[[1e-20 1]
 [1.0 1]]

 [[1e-20 1.0]
 [1.0 0.0]]


Как можно видеть, эти матрицы различаются в последнем элементе. Эта разница обусловлена тем, что в python числа типа float и типа int различаются. Оператор ** переводит число из int во float, поэтому при перемножении мы проводим операции с разными типами данных:

В первом случае - 10^20 * 1 (int) + 1 * 1 - 10^20 (int) -> 10^20 + 1 - 10^20
Операции на больших числах работают лучше, поэтому погрешности не возникает, и 10^20 просто сокращаются.

Во втором случае - 10^20 * 1. (float, так как int * float = float) + 1. * (1 - 10^20) (float) -> из-за того, что мы проводим операцию между float, возникает погрешность, а 10^-20 конвертируется в 0. 

In [ ]:
print(1 * (1 - 10**20))
print(1. * (1 - 10**20))

-99999999999999999999
-1e+20


Отметим, что в реальных вычислениях матричные элементы почти наверняка с самого начала будут числами с плавающей точкой (а не целыми).

Теперь проверьте, что будет, если вычислить QR-разложение исходной матрицы и перемножить матрицы $Q$ и $R$.

In [ ]:
A = np.matrix([[10**-20, 1], [1, 1]])

q, r = np.linalg.qr(A)

print("Q-matrix:\n", q)
print("\nR-matrix:\n", r)

print("\nQR-matrix:\n", q * r)

Q-matrix:
 [[ 0. -1.]
 [-1.  0.]]

R-matrix:
 [[-1. -1.]
 [ 0. -1.]]

QR-matrix:
 [[0. 1.]
 [1. 1.]]


Данный пример отлично показывает, что для numpy 10 в маленькой степени равен нулю, что может значить, что использование библиотек для таких целей не всегда разумно.

**Выход: LU-разложение с выбором главного элемента (по столбцу)**

Каждый раз ищем максимум в столбце и переставляем соответствующую строку наверх.

$$
\begin{pmatrix}
b_{11} & \dots & b_{1i} & b_{1,i+1} & \dots & b_{1n}\\
 & \ddots & \vdots & \vdots & & \vdots\\
 & & \color{blue}{b_{ii}} & \color{blue}{b_{i,i+1}} & \dots & \color{blue}{b_{in}} \\
 & & b_{i+1,i} & b_{i+1,i+1} & \dots & b_{i+1,n}\\
 & & \vdots & \vdots &  & \vdots \\
 & & \color{green}{b_{ji}} & \color{green}{b_{j,i+1}} & \dots & \color{green}{b_{jn}} \\
 & & \vdots & \vdots & & \vdots\\
\end{pmatrix}\longrightarrow
\begin{pmatrix}
b_{11} & \dots & b_{1i} & b_{1,i+1} & \dots & b_{1n}\\
 & \ddots & \vdots & \vdots & & \vdots\\
 & & \color{green}{b_{ji}} & \color{green}{b_{j,i+1}} & \dots & \color{green}{b_{jn}} \\
 & & b_{i+1,i} & b_{i+1,i+1} & \dots & b_{i+1,n}\\
 & & \vdots & \vdots &  & \vdots \\
 & & \color{blue}{b_{ii}} & \color{blue}{b_{i,i+1}} & \dots & \color{blue}{b_{in}} \\
 & & \vdots & \vdots & & \vdots\\
\end{pmatrix}\longrightarrow
$$
$$
\longrightarrow\begin{pmatrix}
b_{11} & \dots & b_{1i} & b_{1,i+1} & \dots & b_{1n}\\
 & \ddots & \vdots & \vdots & & \vdots\\
 & & \color{green}{b_{ji}} & \color{green}{b_{j,i+1}} & \dots & \color{green}{b_{jn}} \\
 & & 0 & b'_{i+1,i+1} & \dots & b'_{i+1,n}\\
 & & \vdots & \vdots &  & \vdots \\
 & & 0 & b'_{i,i+1} & \dots & b'_{in} \\
 & & \vdots & \vdots & & \vdots
\end{pmatrix}
$$

Наибольший, а не первый ненулевой элемент столбца берётся потому, что чем больше число - тем меньшие погрешности потенциально вносит деление на него.

Что при этом происходит? Перестановка строк матрицы равносильна умножению её слева на матрицу соответствующей перестановки. Таким образом, мы получаем равенство

$$L_nP_nL_{n-1}P_{n-1}\ldots L_2P_2L_1P_1 A = U\qquad\qquad(1)$$

где $L_1,\ldots,L_n$ - некоторые нижнетреугольные матрицы.

**Вопрос:** Ну, и где здесь матрица $L$?!

**Ответ:** Введём новые матрицы

\begin{align*}
L'_n &= L_n\\
L'_{n-1} &= P_nL_nP_{n-1}\\
L'_{n-2} &= P_nP_{n-1}L_{n-1}P_n^{-1}P_{n-1}^{-1}\\
&\ldots\\
L'_1 &= P_nP_{n-1}\ldots P_2L_1P_2^{-1}\ldots P_{n-1}^{-1}P_n^{-1}
\end{align*}

**Упражнение.** Матрицы $L'_i$ тоже нижнетреугольные!

Тогда левая часть (1) перепишется в виде

$$\underbrace{L'_nL'_{n-1}\ldots L'_1}_{:=L^{-1}}\underbrace{P_nP_{n-1}\ldots P_1}_{:=P^{-1}}\cdot A$$

**Итог:** разложение вида
$$A = PLU$$
где $P$ - матрица перестановки.

Функция `scipy.linalg.lu` в Питоне находит именно такое разложение!

Все элементы $L$ не превосходят $1$, так что $||L|| \leqslant 1$. При этом
$$||\Delta A|| \leqslant ||A||\cdot O(\rho \varepsilon_{machine}),$$
где
$$\rho = \frac{\max_{i,j}|u_{ij}|}{\max_{i,j}|a_{ij}|}$$
Это число называется *фактором роста матрицы*.

Но что, если это отношение велико?

**Задание 2 (1 балл)** Сгенерируйте матрицу $500\times500$, имеющую вид

$$
\begin{pmatrix}
1 & 0 & 0 & \cdots & 0 & 0 & 1\\
-1 & 1 & 0 &  &  & 0 & 1\\
-1 & -1 & 1 & 0  &  & 0 & 1\\
\vdots & & \ddots & \ddots  & \ddots & \vdots & \vdots \\
-1 & -1 & -1 & \ddots & 1 & 0 & 1\\
-1 & -1 & -1 &  & -1 & 1 & 1\\
-1 & -1 & -1 & \cdots & -1 & -1 & 1
\end{pmatrix}
$$

Например, сгенерировать сначала нулевую матрицу нужного размера, а потом заполнить её клетки правильными числами.

Найдите её PLU-разложение и QR-разложение. Убедитесь, что $P = E$. Вычислите $||A - LU||_2$ и $||A - QR||_2$. Чему равен фактор роста матрицы $A$? Сделайте вывод об устойчивости (или не устойчивости) нахождения PLU-разложения.

In [ ]:
A = np.full([500, 500], -1)
A = np.tril(A)
np.fill_diagonal(A, 1)
A[:,-1] = 1

print("Matrix:\n", A)

Matrix:
 [[ 1  0  0 ...  0  0  1]
 [-1  1  0 ...  0  0  1]
 [-1 -1  1 ...  0  0  1]
 ...
 [-1 -1 -1 ...  1  0  1]
 [-1 -1 -1 ... -1  1  1]
 [-1 -1 -1 ... -1 -1  1]]


In [ ]:
p, l, u = sla.lu(A)

print("P-matrix:\n", p)

print("\nP is an identity matrix,", np.array_equal(p, np.eye(500)))

q, r = np.linalg.qr(A)

P-matrix:
 [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]

P is an identity matrix, True


In [ ]:
first = A - l * u
first = np.matrix(first).transpose() * first
first_norm, v = np.linalg.eigh(first)
first_norm = np.sqrt(max(first_norm))

print("||A - LU||2 = ", first_norm)

second = A - q * r
second = np.matrix(second).transpose() * second
second_norm, v = np.linalg.eigh(second)
second_norm = np.sqrt(max(second_norm))

print("||A - QR||2 = ", second_norm)


||A - LU||2 =  1.636695303948071e+150
||A - QR||2 =  319.26596359596954


Как можно увидеть, PLU разложение более устойчиво, чем QR, так как в случае QR разложения погрешность достаточно большая.

К счастью, на практике так редко бывает (чёрт его знает почему). Тем не менее, QR-разложение всё-таки лучше. Теоретическая оценка для ошибки QR-разложения имеет вид

$$||A - QR||_2 \leqslant ||A||_2\cdot O(\varepsilon_{machine})$$

**Задание 3 (1 балл)** Рассмотрим *матрицу Паскаля* $S_n = \left(C_{i + j}^i\right)$ ($i,j = 0,\ldots,n-1$).

Каково её LU-разложение? Выведите формулы для матриц L и U и приведите краткое обоснование прямо в ноутбуке. Не пользуйтесь функцией `scipy.linalg.lu`, чтобы его "угадать": матрица P будет отлична от единичной, и вы получите не то, что хотели.

Каков её определитель? Обязательно обоснуйте ответ.

Напишите функцию `my_pascal(n)`, генерирующую матрицу Паскаля размера $n\times n$.

In [ ]:
def my_pascal(n):
    return sla.pascal(n)
    raise NotImplementedError()


Найдите норму разности $||A - PLU||_2$. Не такая уж и большая, правда?

In [ ]:
A = my_pascal(30)

# Find ||A - PLU||_2 here

Теперь попросим компьютер вычислить определитель матрицы Паскаля $30\times30$ и решить простенькую систему уравнений:

In [ ]:
print(sla.det(A))

# Try to solve a linear system
x = np.ones(30)
b = A.dot(x)
x1 = sla.solve(A, b)
print(sla.norm(x1 - x))

Так себе ошибка. Теперь попробуем сделать это с помощью QR-разложения. Станет ли лучше?

In [ ]:
Q, R = sla.qr(A)
x2 = sla.solve_triangular(R, Q.T.dot(b))
print sla.norm(x2 - x)

Объясните полученные неприятные результаты.